In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import warnings
warnings.filterwarnings("ignore")

/var/folders/qx/2j533bcs0sb_t334s15wjb0w0000gn/T/ipykernel_27139/3509654866.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
df = pd.read_csv('../diabetic_data.csv')
df.head()

,encounter_id,patient_nbr,race,gender,age,weight,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,...,citoglipton,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,readmitted
0,2278392,8222157,Caucasian,Female,[0-10),?,6,25,1,1,...,No,No,No,No,No,No,No,No,No,NO
1,149190,55629189,Caucasian,Female,[10-20),?,1,1,7,3,...,No,Up,No,No,No,No,No,Ch,Yes,>30
2,64410,86047875,AfricanAmerican,Female,[20-30),?,1,1,7,2,...,No,No,No,No,No,No,No,No,Yes,NO
3,500364,82442376,Caucasian,Male,[30-40),?,1,1,7,2,...,No,Up,No,No,No,No,No,Ch,Yes,NO
4,16680,42519267,Caucasian,Male,[40-50),?,1,1,7,1,...,No,Steady,No,No,No,No,No,Ch,Yes,NO


In [3]:
df.shape

(101766, 50)

In [4]:
df = df.drop('encounter_id', axis=1)

In [5]:
df.isnull().sum()

patient_nbr                     0
race                            0
gender                          0
age                             0
weight                          0
admission_type_id               0
discharge_disposition_id        0
admission_source_id             0
time_in_hospital                0
payer_code                      0
medical_specialty               0
num_lab_procedures              0
num_procedures                  0
num_medications                 0
number_outpatient               0
number_emergency                0
number_inpatient                0
diag_1                          0
diag_2                          0
diag_3                          0
number_diagnoses                0
max_glu_serum               96420
A1Cresult                   84748
metformin                       0
repaglinide                     0
nateglinide                     0
chlorpropamide                  0
glimepiride                     0
acetohexamide                   0
glipizide     

In [6]:
# replacing '?' wiht NaN
df.replace(regex=r'\?', value=np.nan, inplace=True)

In [7]:
df.isnull().sum()

patient_nbr                     0
race                         2273
gender                          0
age                             0
weight                      98569
admission_type_id               0
discharge_disposition_id        0
admission_source_id             0
time_in_hospital                0
payer_code                  40256
medical_specialty           49949
num_lab_procedures              0
num_procedures                  0
num_medications                 0
number_outpatient               0
number_emergency                0
number_inpatient                0
diag_1                         21
diag_2                        358
diag_3                       1423
number_diagnoses                0
max_glu_serum               96420
A1Cresult                   84748
metformin                       0
repaglinide                     0
nateglinide                     0
chlorpropamide                  0
glimepiride                     0
acetohexamide                   0
glipizide     

In [8]:
# response variable 'readmitted', <30 -> 0, >30 1
df['readmitted'] = df['readmitted'].replace({'<30': 1, '>30': 0, 'NO': 0})

In [9]:
df.dtypes

patient_nbr                  int64
race                        object
gender                      object
age                         object
weight                      object
admission_type_id            int64
discharge_disposition_id     int64
admission_source_id          int64
time_in_hospital             int64
payer_code                  object
medical_specialty           object
num_lab_procedures           int64
num_procedures               int64
num_medications              int64
number_outpatient            int64
number_emergency             int64
number_inpatient             int64
diag_1                      object
diag_2                      object
diag_3                      object
number_diagnoses             int64
max_glu_serum               object
A1Cresult                   object
metformin                   object
repaglinide                 object
nateglinide                 object
chlorpropamide              object
glimepiride                 object
acetohexamide       

In [10]:
missing_values = df.isnull().sum()

# Create a DataFrame with only the columns containing missing values
missing_data_table = pd.DataFrame({'Column Name': df.columns,
                                   'Missing Values': missing_values[df.columns],
                                   'Missing Percentage': (missing_values[df.columns] / len(df)) * 100})

# Display the missing data table
print(missing_data_table.to_string(index=False))

             Column Name  Missing Values  Missing Percentage
             patient_nbr               0            0.000000
                    race            2273            2.233555
                  gender               0            0.000000
                     age               0            0.000000
                  weight           98569           96.858479
       admission_type_id               0            0.000000
discharge_disposition_id               0            0.000000
     admission_source_id               0            0.000000
        time_in_hospital               0            0.000000
              payer_code           40256           39.557416
       medical_specialty           49949           49.082208
      num_lab_procedures               0            0.000000
          num_procedures               0            0.000000
         num_medications               0            0.000000
       number_outpatient               0            0.000000
        number_emergency

In [11]:
# dropping columns with large number of missing values
df = df.drop(['weight', 'max_glu_serum', 'A1Cresult'], axis=1)

In [12]:
# Some columns have no variations. The variables'examide'and'citoglipton'have only one value.
# These columns are not useful in prediction and can be deleted. Delete the following near zero-variance colums:
# no variations in cols. these cols are not useful in prediction and can be deleted
cols_to_delete = ['repaglinide', 'nateglinide', 'chlorpropamide', 'glimepiride', 'acetohexamide', 'tolbutamide', 'acarbose', 'miglitol', 'troglitazone', 'tolazamide',
                  'examide', 'citoglipton', 'glyburide-metformin', 'glipizide-metformin', 'glimepiride-pioglitazone', 'metformin-rosiglitazone', 'metformin-pioglitazone']
# Drop multiple columns
df.drop(columns=cols_to_delete, inplace=True)
df.columns

Index(['patient_nbr', 'race', 'gender', 'age', 'admission_type_id',
       'discharge_disposition_id', 'admission_source_id', 'time_in_hospital',
       'payer_code', 'medical_specialty', 'num_lab_procedures',
       'num_procedures', 'num_medications', 'number_outpatient',
       'number_emergency', 'number_inpatient', 'diag_1', 'diag_2', 'diag_3',
       'number_diagnoses', 'metformin', 'glipizide', 'glyburide',
       'pioglitazone', 'rosiglitazone', 'insulin', 'change', 'diabetesMed',
       'readmitted'],
      dtype='object')

In [13]:
df.number_outpatient.unique()

array([ 0,  2,  1,  5,  7,  9,  3,  8,  4, 12, 11,  6, 20, 15, 10, 13, 14,
       16, 21, 35, 17, 29, 36, 18, 19, 27, 22, 24, 42, 39, 34, 26, 33, 25,
       23, 28, 37, 38, 40])

In [14]:
df.dropna(inplace=True)

In [15]:
df.shape

(26755, 29)

In [16]:
df.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
patient_nbr,26755.0,5.732865e+07,3.710987e+07,729.0,24251773.5,44012745.0,92301979.5,189365864.0
admission_type_id,26755.0,2.019137e+00,9.466639e-01,1.0,1.0,2.0,3.0,6.0
discharge_disposition_id,26755.0,2.959821e+00,4.327149e+00,1.0,1.0,1.0,3.0,28.0
admission_source_id,26755.0,4.972267e+00,3.522615e+00,1.0,1.0,7.0,7.0,22.0
time_in_hospital,26755.0,4.317922e+00,2.949791e+00,1.0,2.0,4.0,6.0,14.0
num_lab_procedures,26755.0,4.075668e+01,1.996555e+01,1.0,29.0,42.0,54.0,132.0
num_procedures,26755.0,1.475575e+00,1.749729e+00,0.0,0.0,1.0,2.0,6.0
num_medications,26755.0,1.624672e+01,8.609362e+00,1.0,10.0,15.0,20.0,81.0
number_outpatient,26755.0,2.907120e-01,9.987067e-01,0.0,0.0,0.0,0.0,38.0
number_emergency,26755.0,3.117548e-01,1.437214e+00,0.0,0.0,0.0,0.0,76.0


In [17]:
# identify outliers using IQR
def identify_outliers(column):
    q1 = column.quantile(0.25)
    q3 = column.quantile(0.75)
    iqr = q3 - q1
    lower_bound = q1 - 1.5 * iqr
    upper_bound = q3 + 1.5 * iqr
    outliers = column[(column < lower_bound) | (column > upper_bound)]
    return outliers


cols_to_remove_outliers = ['time_in_hospital', 'num_lab_procedures', 'num_procedures', 'num_medications',
                           'number_outpatient', 'number_inpatient', 'number_emergency', 'number_diagnoses']

# Iterate over each numerical column and identify outliers
for column in cols_to_remove_outliers:
    outliers = identify_outliers(df[column])
    print(f"Outliers in {column}:")
    print(outliers)

Outliers in time_in_hospital:
24069     13
24125     14
24530     14
24561     14
24976     14
          ..
99939     14
100134    13
100945    14
101449    13
101458    14
Name: time_in_hospital, Length: 517, dtype: int64
Outliers in num_lab_procedures:
25554      97
27196     106
30204      93
30736      92
32516      94
         ... 
96101      92
96266      97
97593      94
99874      92
100351     96
Name: num_lab_procedures, Length: 64, dtype: int64
Outliers in num_procedures:
26818     6
27008     6
27057     6
27165     6
27291     6
         ..
101575    6
101611    6
101617    6
101676    6
101747    6
Name: num_procedures, Length: 1533, dtype: int64
Outliers in num_medications:
25755     37
27057     39
27100     39
27165     39
27366     39
          ..
100950    46
101061    47
101439    37
101443    40
101619    36
Name: num_medications, Length: 768, dtype: int64
Outliers in number_outpatient:
24090     6
24129     5
24156     2
24176     1
24201     1
         ..
101590 

In [18]:
# identify and remove outliers using IQR
def remove_outliers(df):
    # Iterate over each numerical column and identify outliers
    for column in cols_to_remove_outliers:
        q1 = df[column].quantile(0.25)
        q3 = df[column].quantile(0.75)
        iqr = q3 - q1
        lower_bound = q1 - 1.5 * iqr
        upper_bound = q3 + 1.5 * iqr

        # Filter the DataFrame to exclude outliers
        df = df[(df[column] >= lower_bound) & (df[column] <= upper_bound)]
    return df


# Remove outliers from the DataFrame
df = remove_outliers(df)

In [19]:
df.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
patient_nbr,17266.0,5.565087e+07,3.829391e+07,729.0,23941986.75,43107403.5,92308299.75,189365864.0
admission_type_id,17266.0,1.950539e+00,8.868757e-01,1.0,1.00,2.0,3.00,6.0
discharge_disposition_id,17266.0,2.802097e+00,4.172648e+00,1.0,1.00,1.0,3.00,28.0
admission_source_id,17266.0,4.828912e+00,3.203826e+00,1.0,1.00,7.0,7.00,22.0
time_in_hospital,17266.0,4.052299e+00,2.585439e+00,1.0,2.00,3.0,5.00,12.0
num_lab_procedures,17266.0,4.013611e+01,1.971215e+01,1.0,29.00,42.0,54.00,91.0
num_procedures,17266.0,1.172246e+00,1.385995e+00,0.0,0.00,1.0,2.00,5.0
num_medications,17266.0,1.472483e+01,6.831579e+00,1.0,10.00,14.0,19.00,35.0
number_outpatient,17266.0,0.000000e+00,0.000000e+00,0.0,0.00,0.0,0.00,0.0
number_emergency,17266.0,0.000000e+00,0.000000e+00,0.0,0.00,0.0,0.00,0.0


In [20]:
df.shape

(17266, 29)

In [21]:
# unique patients
len(df.patient_nbr.unique())

14180

In [22]:
# remove duplicates
df.drop_duplicates(['patient_nbr'], keep='first', inplace=True)
len(df)

14180

In [23]:
# convert cat age to num
replace_dict = {'[0-10)': 5,
                '[10-20)': 15,
                '[20-30)': 25,
                '[30-40)': 35,
                '[40-50)': 45,
                '[50-60)': 55,
                '[60-70)': 65,
                '[70-80)': 75,
                '[80-90)': 85,
                '[90-100)': 95}

df['age'] = df['age'].apply(lambda x: replace_dict[x])

In [24]:
cleaned_df = df

# Define the file path where you want to save the CSV file
file_path = 'cleaned_data.csv'

# Save the DataFrame as a CSV file
cleaned_df.to_csv(file_path, index=False)